In [3]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torchvision
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

device = "cuda" if torch.cuda.is_available() else "cpu"
device

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


'cpu'

In [4]:
df = pd.read_csv("./DatasetDinamicoAplanadoHD.csv", delimiter=',')
df


,0,1,2,3,4,5,6,7,8,9,...,3015,3016,3017,3018,3019,3020,3021,3022,3023,3024
0,0.605908,0.368811,1.405228e-07,0.582437,0.367107,-0.002988,0.569147,0.343845,-0.005482,0.560634,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Hola
1,0.388286,0.441984,1.540007e-08,0.405549,0.443995,0.008910,0.419311,0.422017,0.017610,0.427785,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Hola
2,0.350866,0.390109,7.831241e-08,0.377880,0.393085,0.002870,0.400431,0.359283,0.009616,0.414047,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Hola
3,0.361440,0.344226,9.052938e-08,0.386723,0.353294,0.002524,0.405837,0.335413,0.008556,0.417799,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Hola
4,0.396978,0.395976,6.524542e-08,0.415393,0.400766,0.005262,0.429924,0.383524,0.011371,0.439758,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Hola
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,0.547493,0.585984,9.887079e-08,0.537279,0.540472,0.001392,0.543146,0.498927,0.001652,0.550434,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Saludos
238,0.484293,0.654670,1.462254e-07,0.478069,0.591137,0.008677,0.493383,0.542847,0.011696,0.508520,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Saludos
239,0.399164,0.577123,1.116522e-07,0.410267,0.517150,0.003698,0.402526,0.467928,0.003465,0.390446,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Saludos
240,0.520748,0.512742,7.980219e-08,0.521215,0.464445,-0.001755,0.507990,0.435691,-0.003217,0.494594,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Saludos


In [5]:
# Procesamos datos normalizamos y convertimos a matrices de numpy tambien el features y labels
X_dataF = df.iloc[:, :-1]
Y_dataF = df.iloc[:, -1]
# Leemos nuevamente el csv, las clases para no perder al aplicar replace de pandas
dff = pd.read_csv("./DatasetDinamicoAplanadoHD.csv", delimiter=',')
classes = dff.iloc[:,-1].unique()   # para mantener las clases en string 
classes


array(['Hola', 'BuenasTardes', 'BuenosDias', 'Gracias', 'Chau', 'Permiso',
       'Saludos'], dtype=object)

In [6]:
# Valores texto reemplazar por un numero labels
aliasNumber = np.arange(0,len(classes),1) # Genera un vector de numeros en secuencia
Y_dataF.replace(classes, aliasNumber, inplace=True)
# Convertir los dataframes a arrays de numpy
X_data = X_dataF.to_numpy()
Y_data = Y_dataF.to_numpy()
X_data.shape, Y_data.shape, classes.shape, len(classes)


((242, 3024), (242,), (7,), 7)

In [7]:
X_data[0].shape, Y_data     # Numero de caracteristicas de una fila y los labels en numeros

((3024,),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]))

In [8]:
# Dividimos los valores o leemos desde dataframe para entrenar y evaluar
X_train, X_test, Y_train, Y_test = train_test_split(
    X_data, Y_data, test_size=0.2, shuffle=True, random_state=1234)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((193, 3024), (193,), (49, 3024), (49,))

In [9]:
from torch.utils.data import Dataset, DataLoader

# clase Dataset, hereda de la clase `torch.utils.data.Dataset`
class Dataset(torch.utils.data.Dataset):
    # constructor
    def __init__(self, X, Y=None, train=True):
        self.X = torch.from_numpy(X).float()
        self.Y = torch.from_numpy(Y).long()
        self.train = train
    # devolvemos el número de datos en el dataset

    def __len__(self):
        return len(self.X)
    # devolvemos el elemento `ix` del dataset

    def __getitem__(self, ix):
        if self.train:
            return self.X[ix], self.Y[ix]
        return self.X[ix]


In [10]:
dataset = {  # Instanciamos la clase dataset
    'train': Dataset(X_train, Y_train),
    'eval': Dataset(X_test, Y_test),
    'test': Dataset(X_test, Y_test, train=False)
}

dataloader = {  # Creamos los dataloaders definiendo mas los batch_sizr y mesclar en lo posible
    'train': DataLoader(dataset['train'], shuffle=True, batch_size=32),
    'eval': DataLoader(dataset['eval'], shuffle=False, batch_size=32),
    'test': DataLoader(dataset['test'], shuffle=False, batch_size=32)
}


In [11]:
# Vemos las dimensiones de un batch
x, y = next(iter(dataloader['train']))
x.shape, y.shape

(torch.Size([32, 3024]), torch.Size([32]))

In [12]:
def block(n_in, n_out):
    return nn.Sequential(
        nn.Linear(n_in, n_out),
        nn.ReLU(inplace=True)
    )


class MLP(nn.Module):
    def __init__(self, input_size, output_size=len(classes),classes = classes):
        super().__init__()
        self.classes = classes
        self.input_size = input_size
        self.fc1 = block(input_size, 2000)
        self.fc2 = block(2000, 1000)
        self.fc3 = block(1000, 500)
        self.fc4 = block(500, 100)
        self.fc5 = nn.Linear(100, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        return x


In [13]:
# Para acceder a un batch y a las dimensiones de un dato X
Xaux, Yaux = next(iter(dataloader['train']))
Xaux.shape[1]

3024

In [14]:
# Hacemos la pruena con un batch de prueba al modelo
n_in, n_out = Xaux.shape[1], len(classes) # size input, size output
modelMLP = MLP(n_in, n_out)
output = modelMLP(torch.randn(32, n_in))
output.shape

torch.Size([32, 7])

In [15]:
print(len(dataset['train']))
print(len(dataset['eval']))


193
49


In [16]:
def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1, keepdims=True)


def evaluate(model, x):
    model.to(device)
    model.eval()
    y_hat = model(x)
    y_probas = softmax(y_hat)
    # print(f'prababilidad: {y_probas}')
    return torch.argmax(y_probas, axis=1)


In [17]:
def fit(model, dataloader, epochs=100, lr=0.08):
    model.to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    bar = tqdm(range(1, epochs+1))
    l = []
    for epoch in bar:
        model.train()
        train_loss = []
        for batch in dataloader['train']:
            X, Y = batch
            X, Y = X.to(device), Y.to(device)
            y_pred = model(X)
            loss = criterion(y_pred, Y)
            train_loss.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        l.append(np.mean(train_loss))
        model.eval()
        eval_loss = []
        with torch.no_grad():
            for batch in dataloader['eval']:
                X, Y = batch
                X, Y = X.to(device), Y.to(device)
                Y_hat = model(X)
                loss = criterion(Y_hat, Y)
                eval_loss.append(loss.item())
        # if not e % log_each:
                # print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        bar.set_description(
            f"perdida datos train {np.mean(train_loss):.5f} ===>  perdida datos eval: {np.mean(eval_loss):.5f}")


In [18]:
# Para acceder a un batch y a las dimensiones de un dato X
Xaux, Yaux = next(iter(dataloader['train']))
Xaux.shape[1]

3024

In [18]:
model = MLP(Xaux.shape[1]) # Recibe las dimensiones de un gesto
fit(model, dataloader, epochs=1000, lr=0.05)


perdida datos train 0.00062 ===>  perdida datos eval: 1.21049: 100%|██████████| 1000/1000 [01:31<00:00, 10.97it/s]


In [19]:
def evaluateAccuracy(model, dataloader):
    # Evaluamos el modelo con los datos de evaluacion 
    model.to(device)
    model.eval()
    with torch.no_grad():
        bar = tqdm(dataloader['eval'])
        test_acc = []
        for sign, labs in bar:
            sign, labs = sign.to(device), labs.to(device)
            y_hat = model(sign)
            # print(f'clase: {y_hat}')
            acc = (torch.argmax(y_hat, axis=1) == labs).sum().item() / len(labs)
            test_acc.append(acc)
    print(f' Precision acc: {np.mean(test_acc):.5f}')

In [20]:
# y_predicha = evaluate(model, torch.from_numpy(X_test).float())
# accuracy_score(Y_test, y_predicha.cpu().numpy())

evaluateAccuracy(model, dataloader) # 0.84615   0.76923

100%|██████████| 1/1 [00:01<00:00,  1.15s/it]

 Precision acc: 0.84615


In [35]:
x = torch.rand(32, Xaux.shape[1])
y = model.cpu()(x)
model.eval()

# exportamos el modelo
torch.onnx.export(model,                     # el modelo
    x,                         # un ejemplo del input
    "./modelOnnx3Class.onnx",              # el nombre del archivo para guardar el modelo
    export_params=True,        # guardar los pesos de la red
    opset_version=10,          # versión de ONNX
    do_constant_folding=True,  # optimizaciones
    input_names = ['input'],   # nombre de los inputs
    output_names = ['output'], # nombre de los outputs
    dynamic_axes={'input' : {0 : 'batch_size'},    # ejes con longitud variable (para poder usar diferentes tamaños de batch)
                'output' : {0 : 'batch_size'}})

# Guardar en un archivo CSV las clases
cls = pd.DataFrame(classes)
cls.to_csv("./modelOnnx3Class.csv", index = False)

In [19]:
import onnxruntime
# Evaluamos el modelo en formaato ONNIX exportado
def onnx_evaluate(model, dataloader): 
    # cargarmos el modelo
    ort_session = onnxruntime.InferenceSession(model)
    bar = tqdm(dataloader['eval'])
    acc = []
    with torch.no_grad():
        for batch in bar:
            X, y = batch
            X, y = X.numpy(), y.numpy()
            # generamos los inputs
            ort_inputs = {ort_session.get_inputs()[0].name: X}
            # extraemos los outputs
            ort_outs = ort_session.run(None, ort_inputs)[0]
            acc.append((y == np.argmax(ort_outs, axis=1)).mean())
            bar.set_description(f"Precision: acc {np.mean(acc):.5f}")

In [21]:
onnx_evaluate("./modelOnnx7Class.onnx", dataloader)

Precision: acc 0.95312: 100%|██████████| 2/2 [00:00<00:00, 130.93it/s]


In [22]:
import onnx

onnx_model = onnx.load("./modelOnnx7Class.onnx")
onnx.checker.check_model(onnx_model)

In [39]:
import onnxruntime as onnxrt

def predecirClaseOnnix(x):
    onnx_session= onnxrt.InferenceSession("./modelOnnx7Class.onnx")
    onnx_inputs= {onnx_session.get_inputs()[0].name:[x]}
    onnx_output = onnx_session.run(['output'], onnx_inputs)
    img_label = onnx_output[0][0]
    f_x = np.exp(img_label)/np.sum(np.exp(img_label))
    index = np.argmax(f_x)
    return f_x[index], classes[index], index
    
proba, clss, ind = predecirClaseOnnix(X_test[10])
proba, clss, ind

(0.9999985, 'Saludos', 6)

In [82]:
# Scripting para exportar modelos

scripted_model = torch.jit.script(model.cpu())
scripted_model.save('./modelHandsDinamicHD.zip')


In [51]:
loaded_model = torch.jit.load('./modelHandsDinamicHD.zip')

y_predLoad = evaluateAccuracy(loaded_model, dataloader)
# accuracy_score(Y_train, y_predLoad.cpu().numpy())


100%|██████████| 1/1 [00:02<00:00,  2.07s/it]

 Precision acc: 0.84615


In [52]:
def predecirClase(loaded_model, x):    # recibe una fila de caracteristicas
    loaded_model.eval()
    y_hat = loaded_model(torch.FloatTensor(x))
    y_probas = torch.exp(y_hat) / torch.exp(y_hat).sum(axis=-1, keepdims=True)
    return torch.argmax(y_probas, axis=0)

index = predecirClase(loaded_model, X_train[45])
classes[index] 

'Chau'

In [86]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_normalized = scaler.fit_transform(X_train.reshape(-1, 1))


In [25]:
import os
import time
import cv2
import pickle
import mediapipe as mp
import numpy as np
import pandas as pd
import torch
import onnxruntime as onnxrt

from mediapipe.python.solutions.face_mesh_connections import FACEMESH_CONTOURS

classes = pd.read_csv("./modelOnnx7Class.csv", delimiter=",")['0']   # Classes

# loaded_model = torch.jit.load('./modelHandsDinamic.zip')  # Cargamos el modelo exportado
loaded_model = torch.jit.load('./modelHandsDinamicHD.zip')  # Cargamos el modelo exportado
loaded_model.eval()

onnx_session= onnxrt.InferenceSession("./modelOnnx7Class.onnx")

# Funcion para predecir 
def predecirClase(x):    # recibe una fila de caracteristicas
    y_hat = loaded_model(torch.FloatTensor(x))
    y_probas = torch.exp(y_hat) / torch.exp(y_hat).sum(axis=-1, keepdims=True)
    return torch.argmax(y_probas, axis=0)

def predecirClaseOnnixF(x):
    onnx_inputs= {onnx_session.get_inputs()[0].name:[x]}
    onnx_output = onnx_session.run(['output'], onnx_inputs)
    img_label = onnx_output[0][0]
    f_x = np.exp(img_label)/np.sum(np.exp(img_label))
    index = np.argmax(f_x)
    return f_x[index], classes[index], index

In [30]:
proba, clss, ind = predecirClaseOnnixF(X_test[28])
proba, clss, ind

(0.999992, 'Permiso', 5)

In [40]:
# Esta preparado para detectar solamente manos por ahora las dos palmas de la mano
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

font = cv2.FONT_HERSHEY_TRIPLEX
tipolinea1 = cv2.FILLED
# Características de rectángulo
punto1Rectangulo1 = (500, 650)
punto2Rectangulo1 = (700, 750)
color1 = (0, 0, 0)

# cap = cv2.VideoCapture(0)  # Capturar datos desde la web cam
cap = cv2.VideoCapture("./15fps.mp4")  # Capturar datos desde la web cam


def webcamCaptureSign(longitud=3024, WIDTH = 1280, HEIGHT = 720):
    vectorEntrada = []
    prevTime = 0
    contadorMostrarFrames = 0
    with mp_hands.Hands(
            static_image_mode=False,
            model_complexity = 1,
            min_detection_confidence=0.5,  # Detection Sensitivity
            min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                # vectorEntrada = []
                continue
            frame = cv2.resize(frame, (WIDTH, HEIGHT),interpolation=cv2.INTER_AREA)   # Redimensionar sequendcia
            # image = cv2.flip(image, 1)    # invierte horizontal espejo
            cv2.rectangle(frame, punto1Rectangulo1, punto2Rectangulo1 , color1, tipolinea1)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # BGR image to RGB.
            # To improve performance, optionally mark the image as not writeable topass by reference.
            frame.flags.writeable = False
            results = hands.process(frame)  # pasamos al modelo el frame
            # Draw the hand annotations on the image.
            frame.flags.writeable = True
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # devolvemos a BGR
            image_height, image_width, _ = frame.shape  # Dimensiones del frame

            # Si se encuentran puntos incompletos que no sean 21 0 42 no guarda
            try:
                filaInput = []
                # -----------------------------------------------------
                try:    # Si se detecta los puntos de la mano derecha
                    for j, landmark in enumerate(results.multi_hand_landmarks):  # return 21 o 42 points 1 fps
                            for (i, points) in enumerate(landmark.landmark):
                                # print(i, points)  # indice y el punto en sus 3 coordenadas
                                filaInput.append(points.x)
                                filaInput.append(points.y)
                                filaInput.append(points.z)
                            # Dibuja una mano o dos manos
                            mp_drawing.draw_landmarks(frame, landmark, mp_hands.HAND_CONNECTIONS)
                except:
                    print("No hay ninguna mano")# llenar con ceros si no hay mano right
                    pass
                # -----------------------------------------------------
                if len(filaInput) == 126:  # Si se detecta las dos manos
                    vectorEntrada = np.append(vectorEntrada, filaInput, axis=0)
                    # print("Dos manos a la vez: ",len(vectorEntrada))
                elif len(filaInput) == 63:  # Si no se detecta las dos manos aumentar ceros
                    rowZeros = np.zeros(63)
                    filaInput = np.append(filaInput, rowZeros, axis=0)
                    vectorEntrada = np.append(vectorEntrada, filaInput, axis=0)
                    # print("Una mano a la vez: ",len(vectorEntrada))
                else:
                    print("Vaciando la lista de entrada")
                    vectorEntrada = []
                    contadorMostrarFrames = 0

                if(len(vectorEntrada) == longitud):  # Pasaral model para predecir
                    print("Pasando el vector al modelo: ", len(vectorEntrada))
                    # y_predT = predecirClase(vectorEntrada)
                    proba, clss, ind = predecirClaseOnnixF(X_train[85])
                    putText(frame, clss)
                    # vectorEntrada = []
                    # print("Indice: ", ii, " Clase: ",y_predT)
                    # print("Clase: ", classes[y_predT], " Indice: ",y_predT)
                    vectorEntrada = vectorEntrada[126:] # Eliminar puntos del frame anterior 
                contadorMostrarFrames = contadorMostrarFrames + 1
                print("Frame: ",contadorMostrarFrames)
            except:
                print("Se deberia vaciar la lista que entra al modelo???", len(vectorEntrada))
                vectorEntrada = []
                contadorMostrarFrames = 0
                pass
            
            # mostrat contador de fps
            currTime = time.time()
            fps = 1 / (currTime - prevTime)
            prevTime = currTime
            cv2.putText(frame, f'FPS: {int(fps)}', (20, 70),
                        cv2.FONT_HERSHEY_PLAIN, 3, (0, 245, 54), 2)
            cv2.imshow('MediaPipe Hands', frame)
            if cv2.waitKey(5) & 0xFF == 27:
                break
    cap.release()

def putText(frame, texToFrame):
    cv2.putText(frame, texToFrame, (600, 700), font, 1, (255, 223, 0), 2, cv2.LINE_4)

# Llamamos a la funcion para capturar video por web cam
webcamCaptureSign()

Frame:  1
Frame:  2
Frame:  3
Frame:  4
Frame:  5
Frame:  6
Frame:  7
Frame:  8
Frame:  9
Frame:  10
Frame:  11
Frame:  12
Frame:  13
Frame:  14
Frame:  15
Frame:  16
Frame:  17
Frame:  18
Frame:  19
Frame:  20
Frame:  21
Frame:  22
Frame:  23
Pasando el vector al modelo:  3024
Frame:  24
Pasando el vector al modelo:  3024
Frame:  25
Pasando el vector al modelo:  3024
Frame:  26
Pasando el vector al modelo:  3024
Frame:  27
Pasando el vector al modelo:  3024
Frame:  28
Pasando el vector al modelo:  3024
Frame:  29
Pasando el vector al modelo:  3024
Frame:  30
Pasando el vector al modelo:  3024
Frame:  31
Pasando el vector al modelo:  3024
Frame:  32
Pasando el vector al modelo:  3024
Frame:  33
Pasando el vector al modelo:  3024
Frame:  34
Pasando el vector al modelo:  3024
Frame:  35
Pasando el vector al modelo:  3024
Frame:  36
No hay ninguna mano
Vaciando la lista de entrada
Frame:  1
No hay ninguna mano
Vaciando la lista de entrada
Frame:  1
No hay ninguna mano
Vaciando la lista de

KeyboardInterrupt: 

In [42]:
cap.release()
cv2.destroyAllWindows()


In [59]:
import os
import time
import cv2
import pickle
import mediapipe as mp
import numpy as np
import pandas as pd
import torch

from mediapipe.python.solutions.face_mesh_connections import FACEMESH_CONTOURS
# Funcion para predecir 
def predecirClase(loaded_model, x):    # recibe una fila de caracteristicas
    loaded_model.eval()
    y_hat = loaded_model(torch.FloatTensor(x))
    # y_probas = softmax(y_hat)
    y_probas = torch.exp(y_hat) / torch.exp(y_hat).sum(axis=-1, keepdims=True)
    definal = torch.argmax(y_probas, axis=0)
    return definal

# loaded_model = torch.jit.load('./modelHandsDinamic.zip')  # Cargamos el modelo exportado
loaded_model = torch.jit.load('./modelHandsDinamic.zip')  # Cargamos el modelo exportado
loaded_model

RecursiveScriptModule(
  original_name=MLP
  (fc1): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Linear)
    (1): RecursiveScriptModule(original_name=ReLU)
  )
  (fc2): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Linear)
    (1): RecursiveScriptModule(original_name=ReLU)
  )
  (fc3): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Linear)
    (1): RecursiveScriptModule(original_name=ReLU)
  )
  (fc4): RecursiveScriptModule(original_name=Linear)
)

In [ ]:
# Esta preparado para detectar solamente manos por ahora las dos palmas de la mano
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

font = cv2.FONT_HERSHEY_TRIPLEX
cap = cv2.VideoCapture(0)  # Capturar datos desde la web cam
WIDTH  = 620
HEIGHT = 346

def webcam():
    prevTime = 0
    with mp_hands.Hands(
            min_detection_confidence=0.5,  # Detection Sensitivity
            min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.
                continue
            # Redimensionar sequendcia
            # frame = cv2.resize(frame, (WIDTH, HEIGHT),interpolation=cv2.INTER_AREA)
            # image = cv2.flip(image, 1)    # invierte horizontal espejo
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # BGR image to RGB.
            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            frame.flags.writeable = False
            results = hands.process(frame)  # pasamos al modelo el frame
            # Print handedness and draw hand landmarks on the image.
            # print('Handedness:', results.multi_handedness)

            # Draw the hand annotations on the image.
            frame.flags.writeable = True
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # devolvemos a BGR
            image_height, image_width, _ = frame.shape  # Dimensiones del frame

            if results.multi_hand_landmarks:
                filaInpunt = []
                for j, hand_landmarks in enumerate(results.multi_hand_landmarks):
                    # Recorremos los 21 puntos de una o dos manos
                    for (i, points) in enumerate(hand_landmarks.landmark):
                        # print(i, points)  # indice y el punto en sus 3 coordenadas
                        x = points.x
                        y = points.y
                        z = points.z
                        filaInpunt.append(x)
                        filaInpunt.append(y)
                        filaInpunt.append(z)
                    mp_drawing.draw_landmarks(
                        frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                # print("Dimensiones para la entrada", len(filaInpunt))

                if len(filaInpunt) == 126:  # Si se detecta las dos manos
                    # print("vacio")
                    clase = np.array(predecirClase(loaded_model, filaInpunt))
                    # print("Clase: ", classes[clase])
                    texToFrame = f'{classes[clase]}'
                    cv2.putText(frame,  
                        texToFrame,  
                        (500, 700),  
                        font, 1,  
                        (0, 255, 255),  
                        2,  
                        cv2.LINE_4)
                else:  # Si no se detecta las dos manos aumentar ceros
                    rowZeros = np.zeros(63)
                    filaInpunt = np.append(filaInpunt, rowZeros, axis=0)
                    clase = np.array(predecirClase(loaded_model, filaInpunt))
                    # print("Clase: ", classes[clase])
                    texToFrame = f'{classes[clase]}'
                    cv2.putText(frame,  
                        texToFrame,  
                        (500, 700),  
                        font, 1,  
                        (0, 255, 255),  
                        2,  
                        cv2.LINE_4)

            currTime = time.time()
            fps = 1 / (currTime - prevTime)
            prevTime = currTime
            cv2.putText(frame, f'FPS: {int(fps)}', (20, 70),
                        cv2.FONT_HERSHEY_PLAIN, 3, (0, 196, 255), 2)
            cv2.imshow('MediaPipe Hands', frame)
            if cv2.waitKey(5) & 0xFF == 27:
                break
    cap.release()


# Llamamos a la funcion para capturar video por web cam
webcam()


In [17]:
cap.release()
cv2.destroyAllWindows()